In [1]:
import amazon_dataset
import movielens_dataset
import feature_extraction_text

import numpy as np

For Amazon Dataset

In [4]:
dataset = 'Musical_Instruments'
items = amazon_dataset.items_df(dataset)
items.sample(n=3)


,asin,description,title,brand,main_cat,rank,price,image_slug,image_url,feature,category,tech_detail,item_id
id,,,,,,,,,,,,,
115996,B01D11CBK6,\nThe <b>Reynolds Premium Guitar Accessories C...,Reynolds Guitar Accessories - Guitar Capo - Wi...,Reynolds Guitar Accessories,Musical Instruments,">#112,066 in Musical Instruments (See Top 100 ...",,[41fp0Ep6TWL],[https://m.media-amazon.com/images/I/41fp0Ep6T...,[&#10004; FRET SAFE - High quality silicone pa...,"[Musical Instruments, Instrument Accessories, ...",None,B01D11CBK6
79395,B00FKXZE5I,The Westone UM Pro20 is a professional tool pa...,Westone - Old Model - UM Pro20 High Performanc...,Westone,Musical Instruments,">#71,668 in Musical Instruments (See Top 100 i...",,[41i9skDhgdL],[https://m.media-amazon.com/images/I/41i9skDhg...,[Professional IEM with the range and output fo...,"[Musical Instruments, Studio Recording Equipme...",None,B00FKXZE5I
5522,B0002GJ6FC,Neck Support Stand for Changing Guitar Strings,Planet Waves Guitar Headstand,D'Addario Accessories,Musical Instruments,">#5,589 in Musical Instruments (See Top 100 in...",$9.95,[41W1OjrA4wL],[https://m.media-amazon.com/images/I/41W1OjrA4...,[Securely supports guitars and basses for hass...,"[Musical Instruments, Instrument Accessories, ...",None,B0002GJ6FC


In [ ]:
extracted_bert_features = feature_extraction_text.extract_bert_text_features(items)

In [10]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_bert_features.npz',
    **extracted_bert_features
)

In [11]:
extacted_clip_features = feature_extraction_text.extract_clip_text_features(
    items
)

100%|██████████| 8.46k/8.46k [00:47<00:00, 180product/s]


In [12]:
np.savez_compressed(
    amazon_dataset.BASE_DATA_FOLDER / f'{dataset}_cliptext_features.npz',
    **extacted_clip_features
)

For the Movielens dataset

In [3]:
items = movielens_dataset.items_df()
items.sample(n=3)

,movie_id,title,genres,year,original_title,imdb_id,synopsis,description,image_url,item_id
2516,2585,"Lovers of the Arctic Circle, The (Los Amantes ...","[Drama, Romance]",1998,NaN,133363,None,"In cold Lapland Finland, under the eternal mid...",https://m.media-amazon.com/images/M/MV5BNTM0Nz...,2585
1628,1674,Witness,"[Drama, Romance, Thriller]",1985,NaN,90329,"In 1984, an Amish community outside Lancaster,...",When a young Amish boy is sole witness to a mu...,https://m.media-amazon.com/images/M/MV5BZWQzYm...,1674
3162,3231,"Saphead, The",[Comedy],1920,NaN,11652,None,The simple-minded son of a rich financier must...,https://m.media-amazon.com/images/M/MV5BZDNiOD...,3231


In [14]:
extracted_bert_features = feature_extraction_text.extract_bert_text_features(items)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0.00/3.88k [00:00<?, ?item/s]/home/igui/src/VBPR/feature_extraction_text.py:69: GuessedAtParserWarning: No parser was explicitly specified,

In [15]:
np.savez_compressed(
    movielens_dataset.BASE_DATA_FOLDER / f'ml_1m_bert_features.npz',
    **extracted_bert_features
)

In [4]:
extacted_clip_features = feature_extraction_text.extract_clip_text_features(
    items
)

  0%|          | 0.00/3.88k [00:00<?, ?product/s]/home/igui/src/VBPR/.venv/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  0%|          | 0.00/3.88k [00:01<?, ?product/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 528.00 MiB (GPU 0; 5.79 GiB total capacity; 3.96 GiB already allocated; 381.38 MiB free; 4.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
np.savez_compressed(
    movielens_dataset.BASE_DATA_FOLDER / f'ml_1m_cliptext_features.npz',
    **extacted_clip_features
)